In [1]:
from keras.models import load_model
# Import all packages

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2
import sys
from keras.preprocessing.image import img_to_array
import tensorflow as tf
import pickle
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline

Using TensorFlow backend.


In [16]:
project_path = "/home/mate/develop/PycharmProjects/GeFace/"

In [ ]:

# load the image
image = cv2.imread(project_path+"p_barbi.jpg")
#image = cv2.imread(project_path+"faces_colored/01/nm0000001_rm124825600_1899-5-10_1968.jpg")
# image = cv2.imread("/home/mate/Pictures/dorka1.jpg")
face_cascade = cv2.CascadeClassifier(project_path+'detector/haarcascade_frontalface_default.xml')

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.1, 5)

bb = []
i = 0
output = image.copy()
for (x,y,w,h) in faces:
    bb.append([w * h, i])
    cv2.rectangle(output,(x,y),(x+w,y+h),(255,0,0),2)
 #   roi_gray = gray[y:y+h, x:x+w]
    roi_color = image[y:y+h, x:x+w]
    i += 1
    print(i)
    break

# pre-process the image for classification
image = cv2.resize(image, (96, 96))
image = image.astype("float") / 255.0
image = img_to_array(image)
image = np.expand_dims(image, axis=0)

# load the trained convolutional neural network from disk, followed
# by the category and color label binarizers, respectively
print("[INFO] loading network...")
model = load_model(project_path+"faces_model/regression.hdf5", custom_objects={"tf": tf})
agesLB = pickle.loads(open(project_path+"faces_model/regression.label", "rb").read())
 
# classify the input image using Keras' multi-output functionality
print("[INFO] classifying image...")
(agesProba ) = model.predict(image)
 
# find indexes of both the category and color outputs with the
# largest probabilities, then determine the corresponding class
# labels
agesIdx = agesProba[0].argmax()

agesLabel = agesLB.classes_[agesIdx]
print(agesLabel)

# draw the category label and color label on the image
agesText = "age: {} ({:.2f}%)".format(str(agesLabel),
	agesProba[0][agesIdx] * 100)


cv2.putText(output, agesText, (10, 25), cv2.FONT_HERSHEY_SIMPLEX,
	0.7, (0, 255, 0), 2)

# display the predictions to the terminal as well
print("[INFO] {}".format(agesText))
# show the probabilities for each of the individual labels
output = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)
# show the output image
# print("[INFO] {} years old".format(label))
plt.imshow(output,)